In [ ]:
### Compress child and token together: 10k
# Coverage: 0.9927927927927928
# Functions survived after addidngg in freq_dict: 2220
# Ended up after compression: 2204
# Compression Ratio: 0.8523115931079496

### Compress child and token together: 20k
# Coverage: 0.992019507869652
# Functions survived after addidngg in freq_dict: 4511
# Ended up after compression: 4475
# Compression Ratio: 0.8282915522161035

### Compress child and token together: 30k
# Coverage: 0.9915953774576017
# Functions survived after addidngg in freq_dict: 6620
# Ended up after compression: 6607
# Compression Ratio: 0.8120006470466895

In [ ]:
import pickle

In [ ]:
filename = "../CodeSearchNet/big_csn_data/CodeSearchNet_Java_Search_Preliminary.pkl"
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()

## Function Definitions

In [8]:
import json
import copy
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
    
def dump_dict(data, name):
    with open(name, 'w', encoding="utf8") as f:
        json.dump(data, f)

In [3]:
import ast
def tree2code(tree, indentation):
    code = ""
    parent = list(tree.keys())[0]
    children = tree[parent]
    if 'type' in tree:
        code = tree['text'] + ' '
        if tree['text'] in [';', '{', '}']:
            code += '\n'
            if tree['text'] == '{':
                indentation += 1
            elif tree['text'] == '}':
                indentation -= 1
            code += '\t'*indentation
        #return code, indentation
    elif len(children)==1 and parent == "literal":
        child = children[0]
        child_text = child["text"]
        #type(ast.literal_eval(child_text))
        try:
            if child_text in ['null', 'true', 'false']:
                code = child_text + " "
            elif type(ast.literal_eval(child_text)) == type("STRING"):
                code = "STRING00 "
            elif type(ast.literal_eval(child_text)) == type(1):
                code = "INT00 "
            elif type(ast.literal_eval(child_text)) == type(1.1):
                code = "FLOAT00 "
        except:
            code = "NUM00 "
    else:
        root = list(tree.keys())[0]
        children = tree[root]
        for child in children:
            t, indentation = tree2code(child, indentation)
            code += t
    return code, indentation

In [4]:
import ast
def get_compressed_func():
    global compression_number
    compression_number += 1
    return "$F"+str(compression_number)

In [5]:
def build_dict(tree):
    var_list = []
    global abstract_code_dict_copy
    global abstract_code_keys
    
    
    if 'text' in tree:
        code = tree["text"] + " "
        #print("**:\t",code, 1)
        return (code, 1, 0, [], None)
    else:
        parent = list(tree.keys())[0]
        children = tree[parent]
        if len(children) == 0:
            return ("", 0, 0, [], None)
        
        code = ""
        compressed_func = None
        leaf_count = 0
        id_count = 0
        children_list = []
        #leaf_list = []
        
        '''
        and not (len(children) == 2 and 'text' in children[1] and children[1]['text'] == ';')\
        and not (len(children) == 3 and 'text' in children[0] and 'text' in children[2] and \
                 ((children[0]['text'] == '(' and children[2]['text'] == ')') or 
                  (children[0]['text'] == '{' and children[2]['text'] == '}') or
                  (children[0]['text'] == '[' and children[2]['text'] == ']')))
        '''
        
        if len(children) > 1:
            for child in children:
                (c, n, i, vl, f) = build_dict(child)
                code += c
                leaf_count += n
                id_count += i
                var_list += [v for v in vl if v not in var_list]
                if f: children_list.append(f)
                #else: leaf_list += code.split()
                
            if (var_list and leaf_count - len(var_list) >= 4) or (not var_list and leaf_count > 1) :
                abstract_code = code.split()
                for i in range(len(abstract_code)):
                    if abstract_code[i] in var_list:
                        abstract_code[i] = "$id{}".format(var_list.index(abstract_code[i]))
                abstract_code = ' '.join(abstract_code)
                
                if abstract_code not in abstract_code_dict:
                    compressed_func = get_compressed_func()
                    abstract_code_dict[abstract_code] = {
                        'count': 0, 
                        'compressed_name': compressed_func,
                        'num_params': len(var_list),
                        'children': children_list,
                        #'leaves': leaf_list
                    }
                else:
                    compressed_func = abstract_code_dict[abstract_code]['compressed_name']
                
                abstract_code_dict[abstract_code]['count'] += 1
#                 children_list.insert(0, compressed_func)
                
            return (code, leaf_count, id_count, var_list, compressed_func)
        
        if parent == "literal":
            child = children[0]
            child_text = child["text"]
            try:
                if child_text in ['null', 'true', 'false']:
                    pass
                elif type(ast.literal_eval(child_text)) == type("STRING"):
                    children[0]["text"] = "STRING00"
                elif type(ast.literal_eval(child_text)) == type(1):
                    children[0]["text"] = "INT00"
                elif type(ast.literal_eval(child_text)) == type(1.1):
                    children[0]["text"] = "FLOAT00"
            except:
                children[0]["text"] = "NUM00"
            code = children[0]["text"] + " "
        elif parent == "primary" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)
            code = text + " "
        elif parent == "variableDeclaratorId" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)
            code = text + " "
        else:
            (code, leaf_count, i, vl, compressed_func) = build_dict(children[0])
            id_count += i
            var_list += [v for v in vl if v not in var_list]
        
        return (code, leaf_count, id_count, var_list, compressed_func)

In [6]:
def compress(tree):
    var_list = []
    global abstract_code_dict_copy
    global abstract_code_keys
    
    if 'text' in tree:
        code = tree["text"] + " "
        #print("**:\t",code, 1)
        return (code, 1, 0, [])
    else:
        parent = list(tree.keys())[0]
        children = tree[parent]
        if len(children) == 0:
            return ("", 0, 0, [])
        
        code = ""
        compressed_func = None
        leaf_count = 0
        id_count = 0
        children_list = []
        
        if len(children) > 1:
            for child in children:
                (c, n, i, vl) = compress(child)
                code += c
                leaf_count += n
                id_count += i
                var_list += [v for v in vl if v not in var_list]
                
            if (var_list and leaf_count - len(var_list) >= 4) or (not var_list and leaf_count > 1) :
                abstract_code = code.split()
                for i in range(len(abstract_code)):
                    if abstract_code[i] in var_list:
                        abstract_code[i] = "$id{}".format(var_list.index(abstract_code[i]))
                abstract_code = ' '.join(abstract_code)
                
                #---------------------------------------------------
                if abstract_code in abstract_code_keys:
                    compressed_func = abstract_code_dict_copy[abstract_code]['compressed_name']
                    compressed_func += ' ( ' +' , '.join(var_list)+' )' if var_list else ''
                    tree[parent] = [{"type": "compressed", "text": compressed_func}]
                #---------------------------------------------------

            return (code, leaf_count, id_count, var_list)
        
        if parent == "literal":
            child = children[0]
            child_text = child["text"]
            try:
                if child_text in ['null', 'true', 'false']:
                    pass
                elif type(ast.literal_eval(child_text)) == type("STRING"):
                    children[0]["text"] = "STRING00"
                elif type(ast.literal_eval(child_text)) == type(1):
                    children[0]["text"] = "INT00"
                elif type(ast.literal_eval(child_text)) == type(1.1):
                    children[0]["text"] = "FLOAT00"
            except:
                children[0]["text"] = "NUM00"
            code = children[0]["text"] + " "
            
        elif parent == "primary" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)
            code = text + " "
        elif parent == "variableDeclaratorId" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)
            code = text + " "
        else:
            (code, leaf_count, i, vl) = compress(children[0])
            id_count += i
            var_list += [v for v in vl if v not in var_list]
        
        return (code, leaf_count, id_count, var_list)

In [9]:
## Loading one training data
tree_data = getJsonData("../test_data/java_train_3.json")

### Building Dictionary

In [6]:
abstract_code_dict = {}
compression_number = 0

In [7]:
i = 0
for data in tree_data:
    if not i%5000: print(i)
    build_dict(data)
    i += 1
print(i)

0
5000
10000
15000
20000
25000
30000


In [8]:
dump_dict(abstract_code_dict, '../test_data/abstract_code_dict.json')

In [14]:
## build freq_dict
freq_dict_original = {}
uncompressed_codes = []

for i in range(len(tree_data)):
    if i%5000 == 0: print("[", i, "]")
    code, _ = tree2code(tree_data[i], 0)
    uncompressed_codes.append({"code": code})
    tokens = code.split()

    for token in tokens:
        if token not in freq_dict_original: freq_dict_original[token] = 0
        freq_dict_original[token] += 1
print("[", i, "]")

[ 0 ]
[ 5000 ]
[ 10000 ]
[ 15000 ]
[ 20000 ]
[ 25000 ]
[ 29999 ]


In [15]:
dump_dict(freq_dict_original, '../test_data/freq_dict.json')

In [17]:
dump_dict(uncompressed_codes, "../test_data/uncompressed_codes_no_literal_train_3.json")

In [11]:
abstract_code_dict = getJsonData('../test_data/abstract_code_dict.json')

In [13]:
freq_dict_original = getJsonData('../test_data/freq_dict.json')

In [12]:
func2abs = {}

for key in abstract_code_dict:
    func2abs[abstract_code_dict[key]['compressed_name']] = key

### For Reruns, run from here

In [37]:
vocab_size = 20000

In [38]:
import copy
abstract_code_dict_copy = copy.deepcopy(abstract_code_dict)
abstract_code_keys = sorted(abstract_code_dict_copy, key = lambda x:abstract_code_dict_copy[x]['count'], reverse=True)
len(abstract_code_keys)

411762

In [39]:
freq_dict = copy.deepcopy(freq_dict_original)
most_freq = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)
len(most_freq)

82769

In [41]:
min_freq = freq_dict[most_freq[vocab_size]]
most_freq[vocab_size], min_freq
# 10k : ('nColors', 11)
# 30k: ('ceMask_', 4)
# 20k: ('beginCancelWithServiceResponseAsync', 6)

('beginCancelWithServiceResponseAsync', 6)

In [43]:
# Tokens that are less than the min-count, will not appear on the final vocab

for key in list(abstract_code_dict_copy):
    if abstract_code_dict_copy[key]['count'] < min_freq:
        del abstract_code_dict_copy[key]
abstract_code_keys = sorted(abstract_code_dict_copy, key = lambda x:abstract_code_dict_copy[x]['count'], reverse=True)
len(abstract_code_keys)

11167

In [44]:
## We take one compressed token at a time and remove the tokens it compresses

i = 0
accepted_functions = []
for key in abstract_code_keys:
    if not i%500: print(i, ":", len(accepted_functions))
    i+=1
    func = abstract_code_dict_copy[key]['compressed_name']
    freq_dict[func] = abstract_code_dict_copy[key]['count']
    most_freq = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)
    accepted_functions = list(set(func2abs) & set(most_freq[:vocab_size]))
    
    if func in most_freq[:vocab_size]:
        count = abstract_code_dict_copy[key]['count']
        children = abstract_code_dict_copy[key]['children']
        accepted_functions.append(func)
        tokens = key.split()
        
        # Should be recursive
        if children:
            for child in children:
                if child in freq_dict:
                    freq_dict[child] -= count
                else:
                    print(child, "is less frequent than parent", func)
                    
        else:
            for token in tokens:
                if not token.startswith("$id") and token in freq_dict:
                    freq_dict[token] -= count

most_freq = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)
accepted_functions = list(set(func2abs) & set(most_freq[:vocab_size]))
print(i, ":", len(accepted_functions))

0 : 0
$F356864 is less frequent than parent $F356865
$F355549 is less frequent than parent $F355550
$F355283 is less frequent than parent $F355287
$F11431 is less frequent than parent $F11432
500 : 370
$F113753 is less frequent than parent $F113754
1000 : 732
1500 : 1072
2000 : 1386
2500 : 1712
$F113618 is less frequent than parent $F113619
3000 : 2034
3500 : 2365
$F28782 is less frequent than parent $F28783
4000 : 2654
4500 : 2964
$F113672 is less frequent than parent $F113673
5000 : 3271
5500 : 3556
6000 : 3810
6500 : 4113
7000 : 4396
7500 : 4511
8000 : 4511
8500 : 4511
9000 : 4511
9500 : 4511
10000 : 4511
10500 : 4511
11000 : 4511
11167 : 4511


In [45]:
len(accepted_functions)

4511

In [46]:
freq_dict[most_freq[vocab_size]]

7

In [47]:
to_be_compressed = []
for token in accepted_functions:
    to_be_compressed.append(func2abs[token])

In [48]:
len(to_be_compressed), to_be_compressed[0]

(4511, '< DeploymentExtendedInner >')

In [49]:
del abstract_code_keys
abstract_code_keys = to_be_compressed

In [50]:
len(abstract_code_keys)

4511

In [51]:
dump_dict(to_be_compressed, "to_be_compressed_20k.json")

## Compressing and finding out the overlap between our calculation and actual most frequent

In [29]:
abstract_code_keys = getJsonData("to_be_compressed_30k.json")

In [52]:
compressed_freq_dict = {}
compressed_codes = []
total_tokens_before = 0
total_tokens_after = 0
for i in range(len(tree_data)):
    try:
        tree_copy = copy.deepcopy(tree_data[i])
        code, _ = tree2code(tree_copy, 0)
        total_tokens_before += len(code.split())
        (code, leaf_count, id_count, var_list) = compress(tree_copy)
        code, _ = tree2code(tree_copy, 0)
    except Exception as e:
        print(e)
        code, _ = tree2code(tree_data[i], 0)
    compressed_code = {"code": code}
    tokens = code.split()
    total_tokens_after += len(tokens)
    for token in tokens:
        if token not in compressed_freq_dict:
            compressed_freq_dict[token] = 0
        compressed_freq_dict[token] += 1
    compressed_codes.append(compressed_code)
    if i%5000 == 0: print("[", i, "] :", total_tokens_after/total_tokens_before) 
print("[", i, "] :", total_tokens_after/total_tokens_before)

[ 0 ] : 0.8095238095238095
[ 5000 ] : 0.8727687278521001
[ 10000 ] : 0.8717349690684839
[ 15000 ] : 0.867491867716024
[ 20000 ] : 0.8722404592020389
[ 25000 ] : 0.858101586745733
[ 29999 ] : 0.8282915522161035


In [53]:
dump_dict(compressed_codes, '../test_data/compressed_codes_train_3_20k.json')

In [54]:
compressed_most_freq = sorted(compressed_freq_dict, key = lambda x:compressed_freq_dict[x], reverse=True)
len(compressed_most_freq)

87086

In [55]:
common_tokens_after_compression = list(set(compressed_most_freq[:vocab_size]) & set(accepted_functions))
len(common_tokens_after_compression)

4475

In [56]:
len(common_tokens_after_compression)/len(accepted_functions)

0.992019507869652

In [57]:
len(accepted_functions)

4511

In [44]:
compressed_most_freq[:30]

[')',
 '(',
 ',',
 ';',
 '.',
 '{',
 '}',
 'NUM00',
 '=',
 'if',
 'public',
 'new',
 'String',
 'return',
 '<',
 '>',
 'null',
 'int',
 '==',
 '+',
 'i',
 '[',
 ']',
 'this',
 'else',
 '!=',
 ':',
 '$F1',
 'final',
 '-']

In [32]:
oov = list(set(accepted_functions) - set(compressed_most_freq[:vocab_size]))
len(oov)

16

In [33]:
for o in oov:
    a = abstract_code_dict[func2abs[o]]
    print(func2abs[o], a)

{ $id0 . ifModifiedSince = null ; } {'count': 22, 'compressed_name': '$F355291', 'num_params': 1, 'children': []}
ServiceResponse < Void > {'count': 584, 'compressed_name': '$F355581', 'num_params': 0, 'children': ['$F19311']}
long $id0 = $id1 . currentTimeMillis ( ) {'count': 67, 'compressed_name': '$F11431', 'num_params': 2, 'children': []}
{ $id0 . lastModified = null ; } {'count': 47, 'compressed_name': '$F356833', 'num_params': 1, 'children': []}
$id0 != null ? $id0 . skip ( ) : null {'count': 17, 'compressed_name': '$F361252', 'num_params': 1, 'children': []}
ServiceResponse < OperationStatus > {'count': 190, 'compressed_name': '$F361296', 'num_params': 0, 'children': ['$F361287']}
boolean $id0 = $id1 . $id0 ( ) {'count': 20, 'compressed_name': '$F92224', 'num_params': 2, 'children': []}
{ $id0 . ifUnmodifiedSince = null ; } {'count': 22, 'compressed_name': '$F355294', 'num_params': 1, 'children': []}
{ $id0 . ocpDate = null ; } {'count': 59, 'compressed_name': '$F355283', 'num_p

In [ ]:
'$F406149' in compressed_freq_dict

In [ ]:
common_tokens_after_compression[0]

# Sub-Tokenization

In [ ]:
def state(c):
    n = ord(c)
    if n>=97 and n<=122: # lower case
        return 1
    elif n>=65 and n<=90: # upper case
        return 2
    elif n>=48 and n<=57: # numbers
        return 3
    elif c.isspace(): # whitespaces
        return 4
    elif c in ['_', '$']: 
        return 5
    elif n < 128:
        return 6
    else:
        return 7

def space_up(s):
    if s is None or s == "":
        return ""
    new_s = s[0]
    for i in range(1,len(s)):
        prev_state = state(s[i-1])
        curr_state = state(s[i])
        if prev_state in [1,2] and curr_state in [3]:
            new_s += " "
        elif prev_state in [1] and curr_state in [2]:
            new_s += " "
        elif prev_state in [3] and curr_state in [1,2]:
            new_s += " "
        elif prev_state in [1,2,3] and curr_state in [5]:
            new_s += " "
        elif prev_state in [5] and curr_state in [1,2,3]:
            new_s += " "
        new_s+=s[i]
    return new_s
